In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import torch

df = pd.read_csv('om500114u_si_002.csv')

df.head()

,No.,s1,s2,type 2,"dEv-a, Au","dEv-a, Ru","BE, Ru, a","BE, Ru, v","Ru-C, a","Ru-C, v",...,"CC, Rh, v",dEv-a (Rh),"BE, Rh, a","BE, Rh, v","Rh-C, a","Rh-C, v","Rh-P, a","Rh-P, v","Rh-Cl, a","Rh-Cl, v"
0,1,H,H,1R,14.16,-83.32,171.74,433.58,2.116,1.795,...,1.326,-40.41,200.69,419.62,2.108,1.824,2.338,2.342,2.409,2.421
1,2,H,Me,1R,39.49,-57.82,168.13,427.25,2.130,1.801,...,1.328,-19.63,190.24,411.17,2.115,1.827,2.336,2.340,2.416,2.424
2,3,H,CF3,1R,14.87,-83.75,181.61,459.45,2.104,1.785,...,1.335,-24.50,235.23,453.82,2.088,1.808,2.347,2.350,2.397,2.411
3,4,H,Et,1R,37.90,-59.76,168.06,428.82,2.130,1.802,...,1.328,-19.41,191.45,411.87,2.115,1.828,2.335,2.341,2.415,2.423
4,5,H,iPr,1R,39.66,-57.85,167.78,430.15,2.134,1.804,...,1.328,-18.17,190.14,412.83,2.122,1.831,2.336,2.341,2.417,2.423


In [3]:
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

target = 'dEv-a, Au'
X = df.drop(columns=[target])
y = df[target]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_tensor = torch.FloatTensor(X_scaled)
y_tensor = torch.FloatTensor(y.values).view(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

class DeepRegression(nn.Module):
    def __init__(self, input_dim):
        super(DeepRegression, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.network(x)  
    
model = DeepRegression(X_train.shape[1])
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 200
for epoch in range(num_epochs):
    #Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    
    #Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 20 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

#Evaluate
model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
    test_loss = criterion(test_outputs, y_test)
    print(f'Test Loss: {test_loss.item():.4f}')

ValueError: could not convert string to float: 'H'